# Условие

Восстановить коэффициенты $a,b,c$ функции $f(x)$

$$
f(x)=((a+\epsilon_a)\sin x+(b+\epsilon_b)\ln x)^2+(c+\epsilon_c)x^2
$$

где $\epsilon_i$ случайные велечины в диапазоне $[-0.001,0.001]$

# Решение

Раскроем скобки, проигнорировав $\epsilon$. $f(x)=a^2\sin^2 x+2ab\sin x\ln x+b^2\ln^2 x+cx^2$. Получили многомерную линейную регрессию, потому что функция выражена как сумма слагаемых. Если бы $\epsilon$ были взяты в учет, при раскрытии скобок получилось бы еще одно слагаемое, не связанное с коэффициентами. Оно было бы свободным коэффициентом, который предсказывать не нужно, а вообще, это выглядит как шум.
<br>
Таким образом имеем признаки (фичи) $\sin^2 x,2\sin x\ln x,\ln^2 x,x^2$, а вектор весов $w=(a^2,ab,b^2,c)$.

# Код

## 1. Моя реализация

In [15]:
import numpy as np

np.random.seed(42)
data = np.genfromtxt('184.csv', delimiter=',')
np.random.shuffle(data)
#data = np.array([[x, x * 2] for x in range(1,1001)])

In [16]:
X = np.vstack((np.sin(data[:, 0])**2, 2 * np.sin(data[:, 0]) * np.log(data[:, 0]), np.log(data[:, 0])**2, data[:, 0]**2, np.ones(1000))).T
Y = data[:, 1].reshape(-1, 1)

In [41]:
class LinearModel:
    def __init__(self, in_channels, out_channels, lr=1e-2):
        self.w = np.random.rand(in_channels, out_channels)
        self.lr = lr
    def forward(self, X):
        return X @ self.w
    def backward(self, grad):
        self.w -= self.lr * grad
    def update_lr(self, steps):
        #return
        self.lr *= 1.5

In [10]:
class MSE:
    def loss(Y, Y_pred):
        return np.mean((Y_pred - Y)**2)
    def grad(X, Y, Y_pred):
        return 2 * X.T @ (Y_pred - Y) / Y.shape[0]

In [11]:
def batches(X, Y, size):
    for i in range(0, X.shape[0], size):
        yield X[i:min(i+size, X.shape[0])], Y[i:min(i+size, X.shape[0])]

In [58]:
def train(model, X, Y, loss_fn):
    loss = float('inf')
    for i in range(100000):
        if i % 1000 == 0:
            print(loss)
            if i < 15000:
                model.update_lr(i)
        for Xb, Yb in batches(X, Y, 100):
            Y_pred = model.forward(Xb)
            loss = loss_fn.loss(Yb, Y_pred)
            grad = loss_fn.grad(Xb, Yb, Y_pred)
            model.backward(grad)

In [59]:
model = LinearModel(X.shape[1], Y.shape[1], 1e-6)
train(model, X, Y, MSE)

inf
197.6132103580427
149.75337134299903
100.88735668267462
58.759235105574355
29.65333605051883
14.1847551225722
7.96673094362232
5.994573832897293
5.060408531609547
4.058508598580923
2.9899870533132975
1.9489352337288628
1.039692380414802
0.40940492958407887
0.10715990486937331
0.02646862034776018
0.006881696264380135
0.0020998938116735025
0.0009192587074528247
0.0006213254523617372
0.0005430520445477145
0.0005210356767854788
0.0005141873942138257
0.0005117807523894399
0.0005108295920089049
0.0005104180951971133
0.0005102293946744571
0.0005101399351829748
0.0005100967670526102
0.000510075747149709
0.0005100654653561834
0.0005100604247247405
0.0005100579508136309
0.0005100567359681185
0.000510056139243339
0.0005100558460954895
0.0005100557020746165
0.0005100556313160763
0.0005100555965509413
0.000510055579470484
0.0005100555710782851
0.0005100555669551228
0.0005100555649300928
0.0005100555639338442
0.0005100555634451749
0.0005100555632040775
0.0005100555630871046
0.0005100555630285326

In [60]:
print(model.w)

[[ 9.87147950e+00]
 [ 8.53985233e+00]
 [ 7.38998380e+00]
 [ 3.99976613e+00]
 [-8.91880089e-04]]


In [62]:
print(model.w[0][0]**0.5, model.w[2][0]**0.5, model.w[3][0])

3.1418910699580422 2.7184524635082385 3.9997661343311006


## 2. Реализация через sklearn

In [63]:
from sklearn import linear_model as lm

linearModel = lm.LinearRegression()
linearModel.fit(X, Y)

coeffs = linearModel.coef_
print(coeffs, linearModel.intercept_)

a = coeffs[0][0]**0.5
b = coeffs[0][2]**0.5
c = coeffs[0][3]
print(a, b, c)

[[9.87046009 8.53989328 7.38985564 3.99995786 0.        ]] [-0.00070055]
3.1417288382155943 2.71842889252889 3.9999578643356024
